In [2]:
!pip install faiss-cpu

  Using cached faiss_cpu-1.9.0-cp39-cp39-win_amd64.whl.metadata (4.5 kB)
Using cached faiss_cpu-1.9.0-cp39-cp39-win_amd64.whl (14.9 MB)


In [3]:
# 상병명 데이터 (매복치 관련)
disease_names = [
    "상악 절치의 매복",
    "하악 절치의 매복",
    "상악 견치의 매복",
    "하악 견치의 매복",
    "상악 소구치의 매복",
    "하악 소구치의 매복",
    "상악 제3대구치의 매복",
    "하악 제3대구치의 매복",
    "과잉 매복치",
    "상세불명의 매복치",
]

In [5]:
from transformers import AutoTokenizer, AutoModel
import torch
import faiss

# KoELECTRA 모델 및 토크나이저 로드
model_name = "beomi/koelectra-base-v3-discriminator"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)


# 상병명 텍스트를 KoELECTRA로 임베딩
def get_embeddings(texts):
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    with torch.no_grad():
        outputs = model(**inputs)
    # [CLS] 토큰에 대한 벡터만 사용
    return outputs.last_hidden_state[:, 0, :].numpy()


# disease_names에 있는 상병명을 KoELECTRA로 벡터화
disease_embeddings = get_embeddings(disease_names)

# FAISS 인덱스 생성 (L2 거리 기반)
dimension = disease_embeddings.shape[1]  # KoELECTRA 임베딩 차원
index = faiss.IndexFlatL2(dimension)

# 상병명 벡터를 인덱스에 추가
index.add(disease_embeddings)

OSError: beomi/koelectra-base-v3-discriminator is not a local folder and is not a valid model identifier listed on 'https://huggingface.co/models'
If this is a private repository, make sure to pass a token having permission to this repo either by logging in with `huggingface-cli login` or by passing `token=<your_token>`

In [ ]:
# 진료 행위에 대한 여러 문장 리스트 (new_text)
new_texts = [
    "위쪽 앞니가 자꾸 아픈데, 잇몸 속에 치아가 묻혀 있는 것 같아요. 통증이 너무 심해요.",
    "치과에서 위쪽 앞니가 매복됐다고 하네요. 발치를 해야 한다는데 어떤 치료를 해야 하나요?",
    "위쪽 앞니 주변이 계속 부어오르고 아파요. 이를 빼야 하는 걸까요?",
    "잇몸 속에 앞니가 매복되어 있어서, 씹을 때마다 통증이 느껴집니다. 치료가 필요할 것 같아요.",
    "얼마 전부터 앞니 쪽 잇몸이 붓고, 발치 얘기를 들었는데 어떻게 해야 하나요?",
    "위쪽 앞니가 매복돼서 다른 치아를 밀고 있다고 하던데, 이 상태로 괜찮을까요?",
    "앞니가 매복된 상태로 자꾸 아프고, 잇몸이 빨갛게 부어올라 있어요. 발치가 필요할까요?",
    "위쪽 앞니가 잇몸 속에 묻혀 있다는데, 다른 치아에 영향을 미친다고 해서 걱정돼요.",
    "앞니 주변이 계속 부어있고, 잇몸이 붉어졌어요. 매복치 때문일까요?",
    "잇몸 속에 치아가 묻혀 있어서 발치해야 한다고 들었는데, 수술 후 교정 치료도 해야 할까요?",
]


# 진료 행위 텍스트를 입력받아 상병명 검색 (여러 개의 텍스트 처리)
def search_diseases(texts):
    for text in texts:
        new_embedding = get_embeddings([text])  # 입력된 텍스트를 KoELECTRA로 임베딩
        k = 1  # 가장 가까운 1개의 상병명을 검색
        distances, indices = index.search(new_embedding, k)

        # 가장 가까운 상병명 출력
        closest_index = indices[0][0]
        print(f"입력된 진료 행위: {text}")
        print(f"가장 유사한 상병명: {disease_names[closest_index]}")
        print("-" * 50)


# 테스트: new_texts에 있는 진료 행위들에 대해 상병명 검색
search_diseases(new_texts)